In [374]:
import numpy as np

from numpy import array
from scipy.linalg import expm
from numpy import dot
from numpy import linalg
import plotly.express as px

In [375]:
R2 = np.array([
    [-4.5,  2.0,  1.0,  1.5],
    [ 2.0, -3.5,  0.5,  1.0],
    [ 1.0,  0.5, -2.5,  1.0],
    [ 0.0,  1.0,  2.0, -3.0]
])


Q = array([[-1.4, 0.1, 0.4, 0.9], 
           [4.0, -6.9, 0.9, 2.0], 
           [6.3, 2.0, -11.3, 3.0], 
           [0.7,0.1, 0.2, -1]], dtype=float)
i = np.array([0.25, 0.25, 0.25, 0.25])
test_nst_array1 = np.array([0.3, 0.4, 0.2, 0.1])

Q1 = array([[-0.0935, 0.0148, 0.0558, 0.0229], 
           [0.0469, -0.0676, 0.0108, 0.0099], 
           [0.00, 0.0058, -0.0319, 0.0261], 
           [0.00,0.0132, 0.0370, -0.0501]], dtype=float)




In [376]:
import numpy as np

def calculate_stationary_distribution(Q):
    """
    Calculate the stationary distribution pi for a given substitution rate matrix Q.

    Parameters:
    Q (numpy.ndarray): The substitution rate matrix.

    Returns:
    numpy.ndarray: The stationary distribution pi.
    """
    # Add an additional equation to account for the sum of pi elements being 1
    A = np.vstack([Q.T, np.ones(Q.shape[0])])

    b = np.zeros(Q.shape[0] + 1)
    b[-1] = 1

    # Solve for pi
    pi = np.linalg.lstsq(A, b, rcond=None)[0]

    return pi

def calculate_stationary_rate(Q):
    """
    Calculate the stationary evolution rate mu_stationary for a given substitution rate matrix Q.

    Parameters:
    Q (numpy.ndarray): The substitution rate matrix.

    Returns:
    float: The stationary evolution rate mu_stationary.
    """
    # Get the stationary disitrbution of Q
    pi_stationary = calculate_stationary_distribution(Q)

    # Calculate stationary evolution rate using the formula mu = - sum_i(pi*Qii)
    mu_stationary = - np.sum(pi_stationary*np.diagonal(Q))
    return mu_stationary


In [377]:
def calculate_mu(Q, pi_0, t):
    """
    Calculate the value of mu prime (μ(t)) for a given substitution rate matrix Q,
    initial nucleotide frequency pi_0, and time t.

    Parameters:
    Q (numpy.ndarray): The substitution rate matrix.
    pi_0 (numpy.ndarray): The initial nucleotide frequency distribution.
    t (float): The time at which to calculate μ(t).

    Returns:
    float: The calculated value of μ(t).
    """
    # Calculate f(t) = pi_0 * exp(Qt)
    f_t = dot(pi_0, expm(Q * t))
    
    # Calculate mu'(t) as the sum of the element-wise product of f(t) and the diagonal of Q
    mu = - dot(f_t,np.diagonal(Q))
    
    return mu

In [378]:
def calculate_mu_prime(Q, pi_0, t):
    """
    Correctly calculate the value of mu prime (μ'(t)) based on the provided formula:
    μ'(t) = - pi_0 * Q * exp(Qt) * diag(Q)

    Parameters:
    Q (numpy.ndarray): The substitution rate matrix.
    pi_0 (numpy.ndarray): The initial nucleotide frequency distribution.
    t (float): The time at which to calculate μ'(t).

    Returns:
    numpy.ndarray: The calculated value of μ'(t) as a vector.
    """
    
    # Calculate μ'(t) using the provided formula
    mu_prime_t = -pi_0.dot(Q).dot(expm(Q * t)).dot(np.diagonal(Q))
    
    return mu_prime_t


In [399]:
def mu_mu_prime_range(Q, pi_0, t_range):
    mu_range = []
    mu_prime_range = []
    for t in t_range:
        mu = calculate_mu(Q, pi_0, t)
        mu_prime = calculate_mu_prime(Q, pi_0, t)
        mu_range.append(mu)
        mu_prime_range.append(mu_prime)
    
    return mu_range, mu_prime_range

test_nst_array1 = np.array([0.05, 0.35, 0.35, 0.25])
pi = calculate_stationary_distribution(Q1)
t_range_large =  np.linspace(0, 200, 100)
t_range_small = np.linspace(0, 0.5, 100)
mu_range1, mu_prime_range1 = mu_mu_prime_range(Q1, test_nst_array1, t_range_large)
lim_mu = min(mu_range1)

In [400]:
calculate_mu_prime(R2, pi, 0.5)

0.18762896667202833

In [401]:
px.line(mu_range1)


In [402]:
px.line(mu_prime_range1)